# Kaggle Advance Regression House Price

## Imports

In [1]:
import pandas as pd
from pycaret.regression import *

## Load Data

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train[:10]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
object_columns = train.select_dtypes(include=['object']).columns.tolist()
for i in range(len(object_columns)):
    print(f'{object_columns[i]} -- {train[object_columns[i]].nunique()}')

MSZoning -- 5
Street -- 2
Alley -- 2
LotShape -- 4
LandContour -- 4
Utilities -- 2
LotConfig -- 5
LandSlope -- 3
Neighborhood -- 25
Condition1 -- 9
Condition2 -- 8
BldgType -- 5
HouseStyle -- 8
RoofStyle -- 6
RoofMatl -- 8
Exterior1st -- 15
Exterior2nd -- 16
MasVnrType -- 4
ExterQual -- 4
ExterCond -- 5
Foundation -- 6
BsmtQual -- 4
BsmtCond -- 4
BsmtExposure -- 4
BsmtFinType1 -- 6
BsmtFinType2 -- 6
Heating -- 6
HeatingQC -- 5
CentralAir -- 2
Electrical -- 5
KitchenQual -- 4
Functional -- 7
FireplaceQu -- 5
GarageType -- 6
GarageFinish -- 3
GarageQual -- 5
GarageCond -- 5
PavedDrive -- 3
PoolQC -- 3
Fence -- 4
MiscFeature -- 4
SaleType -- 9
SaleCondition -- 6


In [6]:
train['SaleType'].nunique()

9

## PyCaret setup

In [28]:
rgn_3 = setup(train, target='SalePrice', 
              ignore_features=['Id'], 
              normalize=True,
              remove_multicollinearity=True, 
              multicollinearity_threshold=0.95,
              remove_outliers=True,
              transform_target=True,
              
             )

best_models = compare_models()

,Description,Value
0,Session id,2478
1,Target,SalePrice
2,Target type,Regression
3,Original data shape,"(1460, 81)"
4,Transformed data shape,"(1409, 274)"
5,Transformed train set shape,"(970, 274)"
6,Transformed test set shape,"(439, 274)"
7,Ignore features,1
8,Ordinal features,3
9,Numeric features,36


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0398,0.0046,0.0615,0.8967,0.0063,0.0046,0.8040
gbr,Gradient Boosting Regressor,0.0428,0.0049,0.0639,0.8869,0.0066,0.0050,0.1320
xgboost,Extreme Gradient Boosting,0.0475,0.0059,0.0703,0.8657,0.0073,0.0055,0.1750
rf,Random Forest Regressor,0.0479,0.0058,0.0706,0.8604,0.0073,0.0056,0.1940
et,Extra Trees Regressor,0.0477,0.0062,0.0723,0.8594,0.0075,0.0055,0.2100
br,Bayesian Ridge,0.0432,0.0076,0.0749,0.8369,0.0076,0.0050,0.0740
omp,Orthogonal Matching Pursuit,0.0431,0.0085,0.0772,0.8250,0.0077,0.0050,0.0720
ada,AdaBoost Regressor,0.0586,0.0073,0.0801,0.8230,0.0083,0.0068,0.1020
ridge,Ridge Regression,0.0450,0.0086,0.0792,0.8180,0.0080,0.0052,0.0860
huber,Huber Regressor,0.0451,0.0094,0.0818,0.8071,0.0082,0.0052,0.0790


In [40]:
cbr = create_model('catboost') 
gbr = create_model('gbr')
xgb = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0327,0.0021,0.0461,0.9067,0.0052,0.0042
1,0.0390,0.0031,0.0559,0.9172,0.0058,0.0045
2,0.0560,0.0065,0.0804,0.9288,0.0069,0.0053
3,0.0182,0.0010,0.0311,0.9089,0.0043,0.0028
4,0.0617,0.0120,0.1096,0.7968,0.0106,0.0067
5,0.0383,0.0024,0.0493,0.9234,0.0051,0.0044
6,0.0287,0.0021,0.0455,0.8834,0.0057,0.0040
7,0.0232,0.0010,0.0316,0.9279,0.0039,0.0033
8,0.0624,0.0132,0.1149,0.8451,0.0100,0.0061


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0362,0.0025,0.0497,0.8912,0.0056,0.0046
1,0.0375,0.0028,0.0532,0.9249,0.0055,0.0043
2,0.0565,0.0063,0.0796,0.9303,0.0069,0.0054
3,0.0228,0.0012,0.0352,0.8833,0.0048,0.0036
4,0.0678,0.0132,0.1147,0.7774,0.0111,0.0074
5,0.0415,0.0029,0.0542,0.9075,0.0056,0.0048
6,0.0315,0.0022,0.0471,0.8752,0.0059,0.0044
7,0.0251,0.0010,0.0318,0.9266,0.0040,0.0036
8,0.0673,0.0141,0.1188,0.8346,0.0103,0.0066


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0362,0.0025,0.0501,0.8896,0.0057,0.0046
1,0.0467,0.0044,0.0661,0.8840,0.0069,0.0054
2,0.0731,0.0109,0.1045,0.8797,0.0092,0.0070
3,0.0249,0.0017,0.0412,0.8402,0.0056,0.0039
4,0.0721,0.0144,0.1200,0.7563,0.0117,0.0078
5,0.0418,0.0028,0.0532,0.9107,0.0055,0.0048
6,0.0343,0.0025,0.0503,0.8578,0.0062,0.0048
7,0.0245,0.0011,0.0326,0.9230,0.0041,0.0035
8,0.0717,0.0141,0.1187,0.8349,0.0104,0.0070


In [30]:
bagged_cbr = ensemble_model(cbr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0308,0.0020,0.0446,0.9127,0.0051,0.0039
1,0.0383,0.0032,0.0570,0.9139,0.0059,0.0044
2,0.0574,0.0068,0.0822,0.9257,0.0070,0.0054
3,0.0190,0.0010,0.0323,0.9018,0.0044,0.0030
4,0.0614,0.0116,0.1077,0.8037,0.0105,0.0067
5,0.0373,0.0023,0.0481,0.9270,0.0049,0.0043
6,0.0287,0.0021,0.0454,0.8842,0.0057,0.0040
7,0.0229,0.0010,0.0316,0.9276,0.0040,0.0033
8,0.0630,0.0125,0.1118,0.8533,0.0098,0.0061


In [31]:
print(bagged_cbr)

BaggingRegressor(estimator=<catboost.core.CatBoostRegressor object at 0x177e568c0>,
                 random_state=2478)


In [32]:
bagged_cbr2 = ensemble_model(cbr, n_estimators=50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0314,0.0021,0.0456,0.9086,0.0052,0.0040
1,0.0385,0.0032,0.0570,0.9140,0.0059,0.0045
2,0.0552,0.0065,0.0805,0.9287,0.0069,0.0052
3,0.0189,0.0010,0.0320,0.9037,0.0044,0.0029
4,0.0619,0.0118,0.1085,0.8007,0.0106,0.0067
5,0.0369,0.0023,0.0478,0.9281,0.0049,0.0042
6,0.0287,0.0020,0.0446,0.8880,0.0056,0.0040
7,0.0231,0.0010,0.0316,0.9277,0.0039,0.0033
8,0.0623,0.0122,0.1105,0.8567,0.0096,0.0061


In [39]:
tuned_bagged_cbr = tune_model(cbr, n_iter = 100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0329,0.0023,0.0475,0.9007,0.0054,0.0042
1,0.0360,0.0027,0.0524,0.9272,0.0054,0.0042
2,0.0601,0.0066,0.0815,0.9268,0.0070,0.0057
3,0.0191,0.0009,0.0297,0.9173,0.0041,0.0030
4,0.0648,0.0124,0.1115,0.7895,0.0108,0.0071
5,0.0379,0.0026,0.0506,0.9193,0.0052,0.0043
6,0.0293,0.0023,0.0477,0.8719,0.0059,0.0041
7,0.0248,0.0011,0.0338,0.9170,0.0042,0.0035
8,0.0627,0.0123,0.1110,0.8556,0.0097,0.0061


Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [45]:
stack_1 = stack_models([cbr, gbr])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,72675490581.0196,305346462731196841328640.0000,552581634449.7859,-13425384570840949284929536.0000,4.8140,9218408567.4225
1,5744845618.4074,3366331620240161505280.0000,58020096692.7854,-89299310285234838175744.0000,2.4598,673859997.8071
2,12335020107.5318,10897452660783256174592.0000,104390864833.9655,-119929912483361216004096.0000,3.4235,1116506265.8811
3,15376047921.9369,8503172560073634349056.0000,92212648590.4924,-799223358023808025886720.0000,4.8361,2354794537.3355
4,136010842556.6782,888479371697808792879104.0000,942591837275.1851,-15036884787405900967051264.0000,5.1340,14755084016.2580
5,14897888609.8827,10643783672482871902208.0000,103168714601.2922,-335431152413195473256448.0000,4.1920,1714280522.7618
6,12870556129.3526,8456428877854542921728.0000,91958843391.2396,-475979378706153367142400.0000,4.1982,1789419212.0816
7,84548741180.4842,288881828883434185949184.0000,537477282202.1729,-20917022645427840720306176.0000,5.6526,12042836374.4022
8,323404321733.3968,3269726094200547464708096.0000,1808238395289.8877,-38346414876843190844915712.0000,6.8408,31222701856.2809


In [43]:
blend_1 = blend_models(estimator_list=[cbr, gbr, xgb])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0344,0.0022,0.0472,0.9022,0.0054,0.0044
1,0.0385,0.0031,0.0553,0.9188,0.0057,0.0045
2,0.0584,0.0068,0.0823,0.9254,0.0071,0.0056
3,0.0208,0.0012,0.0345,0.8883,0.0047,0.0032
4,0.0644,0.0126,0.1125,0.7860,0.0109,0.0070
5,0.0377,0.0024,0.0485,0.9258,0.0050,0.0043
6,0.0295,0.0021,0.0457,0.8824,0.0057,0.0041
7,0.0216,0.0008,0.0290,0.9390,0.0036,0.0031
8,0.0614,0.0130,0.1142,0.8471,0.0100,0.0060


In [44]:
blend_tuned = tune_model(blend_1)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0334,0.0021,0.0464,0.9055,0.0053,0.0042
1,0.0392,0.0032,0.0566,0.9150,0.0059,0.0045
2,0.0587,0.0069,0.0830,0.9243,0.0072,0.0056
3,0.0194,0.0011,0.0333,0.8955,0.0046,0.0030
4,0.0632,0.0124,0.1111,0.7909,0.0108,0.0069
5,0.0373,0.0023,0.0476,0.9285,0.0049,0.0043
6,0.0289,0.0021,0.0455,0.8834,0.0057,0.0041
7,0.0216,0.0009,0.0296,0.9367,0.0037,0.0031
8,0.0617,0.0130,0.1139,0.8478,0.0099,0.0060


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [47]:
y_pred = predict_model(blend_tuned, test)

In [48]:
y_pred

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,prediction_label
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,123382.249611
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,152768.530192
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189454.992493
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,198935.417576
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,199026.865442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,81931.717074
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,76677.189916
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,145353.753138
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,121538.301404


In [49]:
submission = y_pred[['Id', 'prediction_label']]
submission.columns = ['Id', 'SalePrice']
submission.to_csv('submission_3.csv', index=False)